# Assignment 4 - Riconoscimento tramite Deep Learning

Requisiti:
- Rifacimento assignment sul riconoscimento, questa volta utilizzando però una rete neurale profonda.

Per questa esercitazioni, le scelte tecniche sono state:
- Yolov8 per la face detection nelle immagini e video
- VGG16 riaddestrata sulle nostre immagini (soltanto negli ultimi layer)

Bonus:
- Utilizzo di una label extra per i volti sconosciuti

In [ ]:
import os
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2 as cv
import pickle, h5py
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Rescaling, Resizing, RandomFlip, RandomRotation, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf

yolo = YOLO('yolov8n-face.pt')

Il primo step consiste nel far scorrere il nostro face detector lungo tutte le immagini già acquisite, fare data augmentation grazie ai layer messi a disposizione da tensorflow e salvare i crop rilevati grazie a Yolo.

Per la data augmentation, in particolare, sono state applicate trasformazioni randomizzate:
- flip orizzontale dell'immagine
- leggera rotazione (fattore 0.2)

In [ ]:
# Inizializziamo le immagini per fare training, test e validazione

train = None
test = None
val = None

Imgs_train = Imgs_test = Imgs_val = None
Labels_train = Labels_test = Labels_val = None

if os.path.exists('train.h5'):
    trainfile = h5py.File("train.h5", "r+")
    Imgs_train = np.array(trainfile["/images"]).astype("float32")
    Labels_train = np.array(trainfile["/meta"]).astype("float32")

    testfile = h5py.File("test.h5", "r+")
    Imgs_test = np.array(testfile["/images"]).astype("float32")
    Labels_test = np.array(testfile["/meta"]).astype("float32")

    valfile = h5py.File("val.h5", "r+")
    Imgs_val = np.array(valfile["/images"]).astype("float32")
    Labels_val = np.array(valfile["/meta"]).astype("float32")

Per le dimensioni di train-test-validation, abbiamo suddiviso le immagini etichettate precedentemente ottenute con le seguenti percentuali, rispettivamente 60-20-20.

In [ ]:
# Processo di face detection dal nostro dataset. le foto sono circa 24k, su queste viene poi fatta data augmentation per arrivare sui 42k foto totali (divise tra insieme di addestramento, test e validazione)

# Facciamo il resize facendo attenzione a mantenere l'aspect ratio per non avere dei crop totalmente distorti
# Il rescaling è stato fatto in modo da mantenere valori tra -1 e 1 piuttosto che tra 0 e 1 per lavorare meglio
# con la relu nel nostro modello, focalizzandoci sui valori maggiori di 0 (come da definizione per la Relu d'altronde)

resize_and_rescale = Sequential([
        Resizing(224, 224, crop_to_aspect_ratio=True),
        Rescaling(scale=1./127.5, offset=-1),
])

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
])

if not os.path.exists('train.h5'):
    root = 'train'
    paths = []
    labels = []
    dirs = ['Davide', 'Gabriele', 'Stefano']

    for dir_ in dirs:
        tmp = glob.glob(f'{root}/{dir_}/*.jpg')
        labels.extend(len(tmp) * [dir_])
        paths.extend(tmp)

    imgs = []
    img_labels = []

    # Sfruttiamo il modello pre allenato di yolo specifico per il rilevamento facciale, nello specifico
    # la versione nano per snellire i tempi

    for i in range(len(paths)):
        path = paths[i]
        img = cv.imread(path)
        aug = data_augmentation(img).numpy()

        result = yolo(img, verbose=False)[0].cpu().numpy()
        result_aug = yolo(aug, verbose=False)[0].cpu().numpy()

        if len(result) != 0 and len(result_aug) != 0:

            boxes = result.boxes
            aug_boxes = result_aug.boxes

            conf = boxes.conf
            aug_conf = aug_boxes.conf

            argmax = np.argmax(conf)
            aug_argmax = np.argmax(aug_conf)

            box = boxes[argmax]
            aug_box = aug_boxes[aug_argmax]

            xyxy = box.xyxy[0]
            xyxy = [round(xy) for xy in xyxy]
            crop = img[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]

            aug_xyxy = aug_box.xyxy[0]
            aug_xyxy = [round(xy) for xy in aug_xyxy]
            aug_crop = img[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]

            # 1. data aug
            # 2. yolo (vado a prendere il quadrato che ingloba sostanzialmente la box di yolo per non deformare la faccia)(oppure preserve aspect ratio di tensorflow.image)
            # 3. crop
            # prima ruotiamo l'immagine, poi passiamo yolo sull'immagine ruotata
            # in questo modo ritagliamo anche un po' di sfondo
            # valutare di ignorare francesco

            crop = resize_and_rescale(crop).numpy()
            aug_crop = resize_and_rescale(crop).numpy()

            imgs.append(crop)
            imgs.append(aug_crop)
            img_labels.append(labels[i])
            img_labels.append(labels[i])

    for path_ in glob.glob(f'{root}/Unknown/*.jpg'):
        img = cv.imread(path)
        result = yolo(img, verbose=False)[0].cpu().numpy()

        if len(result) != 0:

            boxes = result.boxes

            conf = boxes.conf

            argmax = np.argmax(conf)

            box = boxes[argmax]

            xyxy = box.xyxy[0]
            xyxy = [round(xy) for xy in xyxy]
            crop = img[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]
            crop = resize_and_rescale(crop).numpy()

            imgs.append(crop)
            img_labels.append('Unknown')

    # Procediamo a dividere le nostre immagini etichettate in 3 insiemi per fare poi
    # train, test e validation. Le dimensioni di questi saranno rispettivamente il
    # 60%, 20% e 20% del totale delle immagini a disposizione.
    # Salviamo infine tramite pickle il lavoro svolto per evitare sprechi di tempo futuri

    Imgs_train, Imgs_test, Labels_train, Labels_test = train_test_split(imgs, img_labels, test_size=0.4)
    Imgs_val, Imgs_test, Labels_val, Labels_test = train_test_split(Imgs_test, Labels_test, test_size=0.5)
    
    Imgs_train = np.asarray(Imgs_train)
    Imgs_test = np.asarray(Imgs_test)
    Imgs_val = np.asarray(Imgs_val)
    
    Labels_train = LabelEncoder().fit_transform(Labels_train)
    Labels_test = LabelEncoder().fit_transform(Labels_test)
    Labels_val = LabelEncoder().fit_transform(Labels_val)

    file = h5py.File("train.h5", "w")

    train_dataset = file.create_dataset(
        "images", np.shape(Imgs_train), h5py.h5t.IEEE_F32LE, data=Imgs_train
    )
    train_meta_set = file.create_dataset(
        "meta", np.shape(Labels_train), h5py.h5t.STD_U8BE, data=Labels_train
    )
    file.close()  

    file = h5py.File("test.h5", "w")

    train_dataset = file.create_dataset(
        "images", np.shape(Imgs_test), h5py.h5t.IEEE_F32LE, data=Imgs_test
    )
    train_meta_set = file.create_dataset(
        "meta", np.shape(Labels_test), h5py.h5t.STD_U8BE, data=Labels_test
    )
    file.close()  

    file = h5py.File("val.h5", "w")

    train_dataset = file.create_dataset(
        "images", np.shape(Imgs_val), h5py.h5t.IEEE_F32LE, data=Imgs_val
    )
    train_meta_set = file.create_dataset(
        "meta", np.shape(Labels_val), h5py.h5t.STD_U8BE, data=Labels_val
    )
    file.close()  

In [ ]:
imgShape = (224, 224, 3)
num_classes = 4

Sono stati effettuati diversi tentativi al fine di scegliere il modello migliore per il nostro compito, variando sia iperparametri che struttura della rete, avvalendosi anche in alcuni tentativi di regolarizzazione l2 dei pesi in quanto, durante le prime prove, il problema più evidente era un overfitting in fase di test, che si traduceva poi in accuracy estremamente basse in fase di test.

Per evitare sprechi di tempo e risorse, è stato utile aggiungere alla rete un meccanismo di early stopping mediamente sensibile sull'accuracy, nel caso in cui si arrivasse ad una situazione piatta e non riuscisse più ad aumentare dopo alcune epoche. 

La parte più importante è stata sicuramente quella di usare i pesi salvati nei vari checkpoint per riprendere l'addestramento in più parti.

In [6]:
model_name = 'muzio.keras'
h5model = 'muzio.h5'
fine_tune = True

if not os.path.exists(model_name):
  vgg = VGG16(weights='imagenet', include_top=False, input_shape=imgShape)

  for layer in vgg.layers:
    layer.trainable = False

  x = Flatten()(vgg.output)
  prediction = Dense(num_classes, activation='softmax')(x)
  model = Model(inputs=vgg.input, outputs=prediction)

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  reduce_lr = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
  
  early_stop = EarlyStopping(monitor='accuracy', patience = 3, restore_best_weights = True)

  checkpoint = ModelCheckpoint(filepath=h5model, verbose=1, save_best_only=True)

  if fine_tune:
    model.load_weights(h5model)

  hist = model.fit(Imgs_train, 
                Labels_train, 
                epochs=50,
                batch_size=32,
                validation_data=(Imgs_val, Labels_val),
                callbacks=[early_stop, reduce_lr, checkpoint]
                )
    
  model.save('muzio.keras')

  # tentativo per inizializzare i filtri: encoder-decoder, stacchiamo il decoder, aggiungiamo dei dense 

  # suggerimento: inserire layer di dropout in coppia al learning rate (osservare se effettivamente riusciamo a minimizzare la loss)
  # se la loss sale e scende devo abbassare il learning rate
  # altro layer dense intermedio pre classificazione? magari 128
  # più layer conv2d

  # metrics -> val_loss l'importante è che la loss non cresca, la val loss sale e scende perchè non incide su addestramento
  # la train loss deve sempre scendere

  # LR: 0.01, 0.001, 0.0001
else:
  model = load_model(model_name)
  

727/727 [==============================] - ETA: 0s - loss: 0.2334 - accuracy: 0.9311
Epoch 13: val_loss did not improve from 0.22521
727/727 [==============================] - 1053s 1s/step - loss: 0.2334 - accuracy: 0.9311 - val_loss: 0.2473 - val_accuracy: 0.9064 - lr: 3.1623e-04


Estraiamo 20 frame e annotiamoli manualmente per fare model selection: questi 20 frame non andranno a coincidere ovviamente con la porzione del video usata per i test.

In [7]:
classes_ = ['Davide', 'Gabriele', 'Stefano']

annotation = None
font = cv.FONT_HERSHEY_SIMPLEX

if os.path.exists('annotateset.pkl'):
    annotationfile = open('annotateset.pkl', 'rb')
    annotation = pickle.load(annotationfile)
    annotationfile.close()
else:
    annotation = []
    cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
    ANNOTATION_FRAMES = 20
    output_annote = None
    f = 0
    while cap.isOpened() and f < ANNOTATION_FRAMES:
        f += 1
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output_annote is None:
            height, width, channels = frame.shape
            size = (width,height)
            output_annote = cv.VideoWriter('manual.mp4', -1, 2, size)

        faces = yolo(frame, verbose=False)[0]

        faces = faces.numpy()

        frame_faces = []

        intact = frame.copy()

        for box in faces.boxes:
            tmp = intact.copy()
            
            xyxy = box.xyxy[0]
            xyxy = [round(xy) for xy in xyxy]

            tmp = cv.rectangle(tmp, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)

            cv.imshow('', tmp)
            cv.waitKey(0)
            cv.destroyAllWindows()

            # Annoto manualmente ogni bounding box nell'immagine con la vera identità (Davide,Stefano,Gabriele o Unknown)

            frame_face = input()
            frame_faces.append(frame_face)

            frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
            frame = cv.putText(frame, frame_face, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)
        annotation.append(frame_faces)
        output_annote.write(frame)
    cap.release()
    output_annote.release()

    annotatefile = open('annotateset.pkl','wb')
    pickle.dump(annotation, annotatefile)
    annotatefile.close()

In [15]:
cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
MAX_FRAME = 20
i = 0
output = None

test_data = []

while cap.isOpened() and i < MAX_FRAME:
    i += 1
    if True:
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output is None:
            height, width, channels = frame.shape
            size = (width,height)
            output = cv.VideoWriter('deep_auto_test.mp4', -1, 1, size)

        faces = yolo(frame, verbose=False)[0].numpy()

        frame_data = []

        if len(faces) != 0:
            boxes = faces.boxes

            for box in boxes:
                xyxy = box.xyxy[0]
                xyxy = [round(xy) for xy in xyxy]
                crop = np.array([frame[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]])
                crop = resize_and_rescale(crop)

                pred = model.predict(crop, verbose=False)
                conf = np.max(pred)
                class_ = classes_[np.argmax(pred)]
                frame_data.append(class_)

                frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
                frame = cv.putText(frame, class_, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)

            test_data.append(frame_data)

            output.write(frame)
                
cap.release()
output.release()

Calcoliamo l'accuracy del nostro modello: dal momento che il face detector è lo stesso, ci basterà usare come metro di misura la somiglianza delle classi assegnate a ogni volto in ogni frame.

In [16]:
annotation_enc= [LabelEncoder().fit_transform(x) for x in annotation]
test_data_enc = [LabelEncoder().fit_transform(x) for x in test_data]

acc = 0

for annotation_item, test_item in zip(annotation_enc, test_data_enc):
    score = accuracy_score(annotation_item, test_item) 
    acc += score

acc /= len(annotation_enc)

In [17]:
print(acc)

0.15416666666666665


Andiamo adesso ad analizzare un video (che non ha overlap con la porzione usata per la validazione) per testare i risultati effettivi del nostro modello.

In [14]:
cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
MAX_FRAME = 2000
i = 0
output = None

test_data = []

while cap.isOpened() and i < MAX_FRAME:
    i += 1
    if i > 20:
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output is None:
            height, width, channels = frame.shape
            size = (width,height)
            output = cv.VideoWriter('deep_auto.mp4', -1, 15, size)

        faces = yolo(frame, verbose=False)[0].numpy()

        frame_data = []

        if len(faces) != 0:
            boxes = faces.boxes

            for box in boxes:
                xyxy = box.xyxy[0]
                xyxy = [round(xy) for xy in xyxy]
                crop = np.array([frame[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]])
                crop = resize_and_rescale(crop)

                pred = model.predict(crop, verbose=False)
                conf = np.max(pred)
                class_ = classes_[np.argmax(pred)]

                frame_data.append(class_)

                frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
                frame = cv.putText(frame, class_, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)

            test_data.append(frame_data)

            output.write(frame)
                
cap.release()
output.release()

## Misure di accuracy e considerazioni finali

Sono stati testati 7 diversi modelli, con strutture e iperparametri più o meno diversi. Tuttavia i tentativi non hanno portato a risultati con accuracy soddisfacente. Potrebbe essere quindi ragionevole, a valle di tutto il lavoro svolto, dare molto più peso alla fase di raccolta, pulizia e selezione eventuale dei dati, cercando di catturare i volti scelti in condizioni più varie e tenendo conto di fattori come la presenza o meno degli occhiali, dal momento che il modello sembra comportarsi bene nel momento in cui sono stati fatti test di classificazioni usando le immagini di test del dataset di partenza, ma meno bene nel momento in cui le condizioni sono cambiate ed è stato usato un video con un setup leggermente diverso.

Dal momento che ha dimostrato un'accuracy maggiore, andremo ad utilizzare il modello muzio_3.keras

A seguire, la precisione ottenuta e la summary di ogni modello

|   Model   |   Accuracy   |
|-----------|--------------|
|  Muzio 0  |      1%      |
|  Muzio 1  |      23%     |
|  Muzio 2  |      29%     |
|  Muzio 3  |      15%     |
|  Muzio 4  |      7%      |
|  Muzio 5  |      5%      |
|  Muzio 6  |      14%     |
|  Muzio 7  |      37%     |
|  Muzio    |      15%     |

In [ ]:
model_ = load_model('muzio_0.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_1.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_2.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_3.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_4.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_5.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_6.keras')

model_.summary()

In [ ]:
model_ = load_model('muzio_7.keras')

model_.summary()